This kernel have 

**Data augmentation (adding reverted data) :**

    Data:
    Team1, team2, score1, score2, target    
    Additional data:
    Team2, team1, score2, score1, 1 - target

**Prediction on two reversed test df with averaging for stability:**

    Pred = [ f(team1, team2) + f(team2, team1) ] / 2

**Cross validation on any year and train on fixed number of years before the test year**

> Keras was used for example, however it shows stable (top 100 logloss
> score) results for last years competitions using just one hot encoding
> of team number

In [2]:
from keras.layers import Input, Dense, Dropout, Flatten, Embedding, merge
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.advanced_activations import PReLU
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.models import Model
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
#from keras.regularizers import l2, activity_l2
from keras.utils import np_utils, generic_utils

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
import random
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

In [6]:
RegularSeasonDetailedResults = pd.read_csv('RegularSeasonDetailedResults.csv')
TourneyDetailedResults = pd.read_csv('TourneyDetailedResults.csv')
Teams = pd.read_csv('Teams.csv')

In [7]:
#union TourneyDetailedResults and RegularSeasonDetailedResults in one for training
TourneyDetailedResults['Table'] = 'Tourney'
RegularSeasonDetailedResults['Table'] = 'Regular'

union = pd.concat([TourneyDetailedResults,RegularSeasonDetailedResults],axis =0, ignore_index=True)
union=union.reset_index()
del TourneyDetailedResults,RegularSeasonDetailedResults

In [8]:
union.head()

,index,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,...,Lftm,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf,Table
0,0,2003,134,1421,92,1411,84,N,1,32,...,14,31,17,28,16,15,5,0,22,Tourney
1,1,2003,136,1112,80,1436,51,N,0,31,...,7,7,8,26,12,17,10,3,15,Tourney
2,2,2003,136,1113,84,1272,71,N,0,31,...,14,21,20,22,11,12,2,5,18,Tourney
3,3,2003,136,1141,79,1166,73,N,0,29,...,12,17,14,17,20,21,6,6,21,Tourney
4,4,2003,136,1143,76,1301,74,N,1,27,...,15,20,10,26,16,14,5,8,19,Tourney


**Data Augmentation. Adding Reverse Data**

In [ ]:
def test_train_split(year,df,shift=99):
    '''拆分出训练数据集与测试数据集'''
    msk_test_year = df.Season==year
    msk_Tourney = df.Table == 'Tourney'  #锦标赛
    msk_Regular = df.Table == 'Regular'  #常规赛
    
    msk_train_yearmin = df.Season>(year-shift) 
    msk_train_yearmax_soft = df.Season<=year
    msk_train_yearmax = df.Season<year    
    test = df[msk_test_year & msk_Tourney]
    
    train = df[(msk_Tourney & msk_train_yearmax & msk_train_yearmin) | 
                  (msk_Regular & msk_train_yearmax_soft & msk_train_yearmin) ]
    return test, train

In [ ]:
def df_shuffle(df,cols_middle,cols_right,cols_left):
    cols = []
    cols.extend(cols_middle)
    cols.extend(cols_right)
    cols.extend(cols_left)
    
    cols_rev = []
    cols_rev.extend(cols_middle)
    cols_rev.extend(cols_left)
    cols_rev.extend(cols_right)
    
    df_sample = pd.DataFrame()
    df_sample = df[cols].copy()
    
    df_rev = pd.DataFrame()
    df_rev[cols] =df[cols_rev].copy()
    
    msk = np.random.choice(df_sample.index,int(len(df)/2))
    
    df_sample['Score_diff'] = df_sample.Wscore - df_sample.Lscore
    df_rev['Score_diff'] = df_rev.Wscore - df_rev.Lscore
    
    df_sample.loc[msk],df_rev.loc[msk] = df_rev.loc[msk],df_sample.loc[msk]
    return df_sample,df_rev

In [ ]:
col_W = []
col_L = []
col_R =[]
for col in union.columns:
    if (col[0]=='W') & (col!='Wloc'):
        col_W.append(col)
    elif col[0]=='L':
        col_L.append(col)
    else:
        col_R.append(col)

In [ ]:
col_under =[]
for col in union.columns:
    if (col[1]=='_'):
        col_under.append(col)

In [ ]:
#shuffle (switch winning team and loosing team in 50% cases) union df = union_st
#and makу reverse df  = union_rev
random.seed(a=123)
union_st,union_rev = df_shuffle(union,col_R,col_W,col_L)

In [ ]:
#if you want to train only on Tourney data, run:

# union_st = union_st[union_st.Table == 'Tourney']
# union_rev = union_rev[union_rev.Table == 'Tourney']

In [ ]:
#make train and test df. 
year = 2016 #for test df - tourney data from year 2016 
shift = 6 #number of years to use in training
#change these parameters for validation on different year

test, train = test_train_split(year,union_st,shift) 
test_rev, train_rev = test_train_split(year,union_rev,shift) # need predict for two df and average results for stability
train = pd.concat([train,train_rev], axis=0) #train df have augmented data (+ reversed)
del train_rev

In [ ]:
train[['Season','Table','Score_diff']].groupby(['Season','Table']).count()

In [ ]:
test[['Season','Table','Score_diff']].groupby(['Season','Table']).count()

## onehotencoding teams

In [ ]:
teams_encoder = OneHotEncoder(sparse=False)
teams_encoder.fit(Teams[['Team_Id']])

In [ ]:
train_ohe = np.hstack((teams_encoder.transform(train[['Wteam']]),teams_encoder.transform(train[['Lteam']])))
test_ohe = np.hstack((teams_encoder.transform(test[['Wteam']]),teams_encoder.transform(test[['Lteam']])))
test_rev_ohe = np.hstack((teams_encoder.transform(test_rev[['Wteam']]),teams_encoder.transform(test_rev[['Lteam']])))

## Keras model + validation

In [ ]:
target_train = (train.Score_diff>0).astype(int) 
target_test = (test.Score_diff>0).astype(int) 
target_rev_test = (test_rev.Score_diff>0).astype(int) 

In [ ]:
target_train = np_utils.to_categorical(target_train.reset_index(drop=True))
target_test = np_utils.to_categorical(target_test.reset_index(drop=True))
target_rev_test = np_utils.to_categorical(target_rev_test.reset_index(drop=True))

In [ ]:
dims = train_ohe.shape[1]
print(dims, 'dims')

model = Sequential()
model.add(Dense(68, input_shape=(dims,)))
model.add(Dropout(0.2))#    input dropout
model.add(BatchNormalization())
model.add(PReLU())

model.add(Dense(68))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(PReLU())

model.add(Dense(16))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(PReLU())


model.add(Dense(16))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(PReLU())

model.add(Dense(2))
model.add(Activation('softmax'))
#opt=SGD(momentum=0.9)
model.compile(loss='binary_crossentropy', optimizer="sgd")

In [ ]:
print('Training model...')
nb_epoch=100
logloss_scores=[]  
logloss_scores_rev=[]   
logloss_scores_avg=[]  
logloss_scores_tr=[]    
for i in range(nb_epoch):
    model.fit(train_ohe, target_train, nb_epoch=1,batch_size=128,verbose= 0)

    y_pre = model.predict(test_ohe, verbose= 0)
    scores = log_loss(target_test,y_pre)
    logloss_scores.append(scores)

    y_pre_rev = model.predict(test_rev_ohe,verbose= 0)
    scores_rev = log_loss(target_rev_test,y_pre_rev)
    logloss_scores_rev.append(scores_rev)

    scores_avg = log_loss(target_test,(y_pre+1-y_pre_rev)/2)
    logloss_scores_avg.append(scores_avg)

    y_pre_tr = model.predict(train_ohe,verbose= 0)
    scores_tr = log_loss(target_train,y_pre_tr)
    logloss_scores_tr.append(scores_tr)
print('Done')

# 2016
https://www.kaggle.com/c/march-machine-learning-mania-2016/leaderboard

top1   0.481310

top50  0.541537

top100 0.551282

Median Submission Benchmark 0.558719

In [ ]:
#2016 (ohe, 6 years shift, with regular season)

plt.plot(logloss_scores,'b') #test log_loss
plt.plot(logloss_scores_rev,'g') #test log_loss for reversed data
plt.plot(logloss_scores_avg, 'r') #average test log_loss
plt.plot(logloss_scores_tr,'y--')  #train log_loss
plt.ylim((0.45,0.65))
plt.axhline(y= 0.481310, color='gray', linestyle='--') #top 1
plt.axhline(y= 0.541537, color='gray', linestyle='--') #top 50
plt.axhline(y= 0.551282, color='gray', linestyle='--') #top 100